In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
plt.style.use("seaborn")

In [ ]:
class SMABacktester():
    
    '''
    Class for vectorised backtesting of SMA-based trading strategies.
    '''
    
    def __init__(self, ticker, shortwindow, longwindow, start, end):
        
        '''
        Parameters
        
        ticker: str
            The ticker symbol indicating the instrument which will be backtested.
            
        shortwindow: int
            The moving window for the shorter SMA. Bars are daily by default.
        
        longwindow: int
            The moving window for the longer SMA. Bars are daily by default.
        
        start: str
            The start date for the imported data. Format is flexible, but if you wish to 
            play safe, use YYYY-MM-DD.
            
        end: str
            The start date for the imported data. Format is flexible, but if you wish to 
            play safe, use YYYY-MM-DD.
        '''
        self.symbol = symbol
        self.Short_SMA = shortwindow
        self.Long_SMA = longwindow
        self.start = start
        self.end = end
        self.results = None
        
        self.get_data()
        self.prepare_data()
    
    def __repr__(self):
        return f"SMABacktester(ticker = {self.symbol}, shortwindow = {self.Short_SMA}, longwindow = {self.Long_SMA}, start = {self.start}, end = {self.end})"
        
    def get_data(self):
        
        '''
        Imports the data from a given source. Default is forex_pairs.csv .
        '''
        raw = pd.read_csv("forex_pairs.csv"), parse_dates = ['Date'], index_col = 'Date'
        raw[self.symbol].to_frame().dropna()
        raw = raw.loc[self.start:self.end].copy()
        raw.rename(columns = {self.symbol: 'Price'}, inplace = True)
        raw['Returns'] = np.log(raw/raw.shift(1))
        self.data = raw
        #can include return raw if you want to see the dataframe when running get_data
        
    def prepare_data(self):
        
        '''
        Prepares the data for strategy-specific backtesting.
        '''
        data = self.data.copy()
        data['SMA Short'] = data['Price'].rolling(self.Short_SMA).mean()
        data['SMA Long'] = data['Price'].rolling(self.Long_SMA).mean()
        self.data = data
        
    
    def set_parameters(self, shortwindow=None, shortwindow=None):
        
        '''
        Updates SMA parameters, and updates the prepared dataset to reflect the new parameters.
        '''
        if shortwindow is not None:
            self.Short_SMA = shortwindow
            self.data['SMA Short'] = self.data['Price'].rolling(self.Short_SMA).mean()
        if longwindow is not None:
            self.Long_SMA = longwindow
            self.data['SMA Long'] = self.data['Price'].rolling(self.Long_SMA).mean() 
    
    def test_strategy(self):
        
        '''
        Backtests the strategy.
        '''
        data = self.data.copy().dropna()
        #1 is go long, -1 is short
        data['Position'] = np.where(data['SMA Short'] > data['SMA Long'], 1, -1)
        data['Strat Returns'] = data['Position'].shift(1) * data['Returns']
        data.dropna(inplace=True)
        data['Cumulative Returns'] = data['Returns'].cumsum().apply(np.exp) #buy and hold
        data['Cumulative Strat Returns'] = data['Strategy'].cumsum().apply(np.exp) #strategy
        self.results = data
    
        perform = data['Cumulative Strat Returns'].iloc[-1] #absolute performance
        outperform = perform - data['Cumulative Returns'].iloc[-1] #outperformance vs. buy and hold
        return round(perform, 6), round(outperform, 6) #to 6 d.p.
    
    def plot_results(self):
        
        '''
        Plots the performance of the trading strategy, and compares to the benchmark 
        i.e. if you had simply bought and held the instrument.
        '''
        if self.results is None:
            print("Run test_strategy() first.")
        else:
            title = f'{self.symbol} | SMA Short = {self.Short_SMA} | SMA Long = {self.Long_SMA}'
            self.results[['Cumulative Returns', 'Cumulative Strat Returns']].plot(title=title, figsize = (11,7))
    
    def optimise_parameters(self, shortwindow_range, longwindow_range):
        
        '''
        Finds the optimal strategy by finding the global maximum given two SMA parameter ranges.
        
        Parameters
        
        shortwindow_range: tuple
            Tuple of the form (start, end, step size), indicating the range of possible short
            window values. To be plugged into a range function that is used to calculate all
            combinations of possible short and long window values.
        
        longwindow_range: tuple
            Tuple of the form (start, end, step size), indicating the range of possible long
            window values. To be plugged into a range function that is used to calculate all
            combinations of possible short and long window values.
        '''
        combinations = list(product(range(*shortwindow_range), range(*longwindow_range)))
        
        #testing combinations
        results = []
        for comb in combinations:
            self.set_parameters(comb[0], comb[1])
            results.append(self.test_strategy()[0])
        
        best_perform = np.max(results) #best performance
        optimum = combinations[np.argmax(results)] #the parameters of the best performance
        
        #run the optimal strat
        self.set_parameters(optimum[0], optimum[1])
        self.test_strategy()
        
        #create a df with the variety of results
        resultsframe = pd.DataFrame(data = combinations, columns = ['SMA Short', 'SMA Long'])
        resultsframe['Performance'] = results
        self.results_overview = resultsframe
        
        return optimum, best_perform
    
    